In [1]:
import numpy as np
import scipy.stats

In [2]:
sequence = "ACTCGTTAGCTAG"
for position, letter in enumerate(sequence):
    flip = np.random.random()
    mut_rate=10^-10
    if flip < mut_rate:
        nucleotides=["A", "G", "C", "T"]
        nucleotides.remove(letter)
        new_letter = np.random.choice(nucleotides)
        
        sequence = sequence[:position] + new_letter.lower() + sequence[position+1:]
        
sequence

'ACTCGTTAGCTAG'

In [3]:
def mutation(sequence):
    mut_rate=10**-10
    for position, letter in enumerate(sequence):
        flip = np.random.random()
        if flip < mut_rate:
            if letter=='A':
                new_letter=np.random.choice(['g','c','t'], p=[.54,.23,.23])
            elif letter=='G':
                new_letter=np.random.choice(['a','c','t'], p=[.74,.13,.13])
            elif letter=='T':
                new_letter=np.random.choice(['a','c','g'], p=[.23,.54,.23])
            elif letter=='C':
                new_letter=np.random.choice(['a','t','g'], p=[.13,.74,.13])
            sequence = sequence[:position] + new_letter + sequence[position+1:]
    return(sequence)

In [6]:
def mutation(list_of_sequences, Number_of_sequence):"""
    Input: 
    Biological sequence
    Number_of_sequences: How many sequences
    Frequencies: ho
    Output: Mutated biological sequence 
    
    """
    number_of_mutations= int(len(sequence)*(9/10))
    
    for sequence in range(number_of_mutations):
        sequence_change(sequence)
    return(sequence)

IndentationError: unexpected indent (<ipython-input-6-3bc60037ad75>, line 9)

In [ ]:
def sequence_change(sequence):
   
    sequence_array=np.array(list(sequence))
   
    flip=np.random.random()
    # 40% letter mutated is a G
    
    if flip<.40:
        allpositions= list(np.where(sequence_array=='G'))
        chosen_position=np.random.choice(allpositions[0])
        new_letter=np.random.choice(['a','c','t'])
        sequence = sequence[:chosen_position] + new_letter + sequence[chosen_position+1:]
        
    if .40<flip<.8:
        #This is where C would go
        #Write saying search sequences for C and choose random C
        allpositions= list(np.where(sequence_array=='C'))
        chosen_position=np.random.choice(allpositions[0])
        new_letter=np.random.choice(['a','t','g'])                            
        sequence = sequence[:chosen_position] + new_letter + sequence[chosen_position+1:]
        sequence[chosen_position]
        
    if .8<flip<.9:
        #This is where A would go
        #Write saying search sequences for A and choose random A
        allpositions= list(np.where(sequence_array=='A'))
        chosen_position=np.random.choice(allpositions[0])
        new_letter=np.random.choice(['g','c','t'])
        sequence = sequence[:chosen_position] + new_letter + sequence[chosen_position+1:]   
        
    if .9 <flip<1:
        #This is where T would go
        #Write saying search sequences for T and choose random T
        allpositions= list(np.where(sequence_array=='T'))
        chosen_position=np.random.choice(allpositions[0])
        new_letter=np.random.choice(['a','c','g'])
        sequence = sequence[:chosen_position] + new_letter + sequence[chosen_position+1:]
        
    return(sequence)

In [129]:
#Function to update dictionary, 1 generation 
#later not trivial
def mutation_dict(population):
    number = int(np.sum(list(pop.values())))
    number_of_mutations = scipy.stats.binom.rvs(number, 10**-10)
    
    pop_list = list(population)
    sequence = pop_list[0]
    
    for pop_list in range(number_of_mutations):
        population[sequence_change(sequence)] = 1
    
    population[sequence] = (2*number-number_of_mutations)
    
    return(population)
        

In [5]:
 RandSeq1 = "ataggagcgtcatcaaacgcgccgttcaggttctggttctccatgccatagttaagccgcacaacgggtactaccactccctgtagtccgctttaccgttctc".upper()